<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Sentiment Analysis and Naive Bayes

_Authors: Kiefer Katovich (SF)_

---

In the sentiment analysis lesson, we used a predefined dictionary of positive and negative valences for words. This  lab inverts that process: You'll find which words are most likely to appear in positive or negative reviews by using the rotten versus fresh binary label.

### Naive Bayes

A common practical way to do this is with the Naive Bayes algorithm. Naive Bayes classifiers are covered in more depth in another lecture — for this lab, you'll just be leveraging the scikit-learn implementation.

Given a feature, $x_i$, and target, $y_i$, Naive Bayes classifiers solve for $P(x_i \;|\; y_i)$. In other words, they solve for the probability of a feature/predictor _given_ that the target is one.

We'll use this to figure out which words are more likely to appear when the target is one ("fresh") versus zero ("rotten").

---

### 1) Load the packages and movie data.

Perform any necessary cleaning.

In [1]:
import pandas as pd
import numpy as np

# We are using the BernoulliNB version of Naive Bayes, which assumes that predictors are binary encoded.
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
rt = pd.read_csv('./datasets/rt_critics.csv')

In [3]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,fresh,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story
2,David Ansen,fresh,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story
3,Leonard Klady,fresh,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story
4,Jonathan Rosenbaum,fresh,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story


In [4]:
import string
rt['qt'] = rt['quote'].map(lambda x: x.translate(None, string.punctuation).lower())

In [5]:
rt['quote_len'] = rt['quote'].map(lambda x: len(x))

In [6]:
rt = rt[(rt['fresh'] != 'none') & (rt['quote_len']>10)]

---

### 2) Create a predictor matrix of words in the quotes with `CountVectorizer`.

It's up to you to select an n-gram range. **Make sure that `binary=True`**.

In [24]:
#pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score 
pclass = Pipeline([
    ('vect', CountVectorizer(strip_accents='unicode', binary=True, stop_words='english', ngram_range=(1,3), min_df=2)),
#     ('tfidf', TfidfTransformer()),
#     ('cls', MultinomialNB())
#     ('logit', LogisticRegression())
    ('bnb', BernoulliNB())
]) 

In [25]:
from sklearn.model_selection import train_test_split
X = rt['qt']
y = rt['fresh']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42) #can add in stratify=y

In [26]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(pclass, X_train, y_train, cv=5)
scores

array([0.73684211, 0.7676182 , 0.74130241, 0.73806336, 0.75457385])

In [27]:
#fit and predict
class_pred = class_fit.predict(X_test)

In [36]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

conmat = np.array(confusion_matrix(y_test, class_pred))
confusion = pd.DataFrame(conmat, index=['fresh','rotten'], columns=['fresh','rotten'])
print (confusion)
print (classification_report(y_test, class_pred))
metrics.f1_score(y_test, class_pred, average='macro')

        fresh  rotten
fresh    1488     230
rotten    469     615
             precision    recall  f1-score   support

      fresh       0.76      0.87      0.81      1718
     rotten       0.73      0.57      0.64      1084

avg / total       0.75      0.75      0.74      2802



0.7237159996191322

---

### 3) Split the data into training and testing sets.

You should keep 25 percent of the data in the testing set.

In [78]:
from sklearn.model_selection import train_test_split
X2 = X.drop('y',axis=1)
y2 = X['y']
X_train, X_test, y_train, y_test = train_test_split(X2, y2, stratify=y2, test_size=0.2, random_state=42) #can add in stratify=y

---

### 4) Build a `BernoulliNB` model predicting fresh versus rotten from the word occurrences.

The model should only be built (and cross-validated) on the training data.

Cross-validate the score and compare it to the baseline.

In [81]:
y_train.value_counts()/len(y_train)

fresh     0.612955
rotten    0.387045
Name: y, dtype: float64

---

### 5) Pull out the probability of words given "fresh."

The `.feature_log_prob_` attribute of the Naive Bayes model contains the log probabilities of a feature appearing given a target class.

The rows correspond to the class of the target and the columns correspond to the features. The first row is the zero, "rotten" class and the second row is the one, "fresh" class.

#### 5.A) Pull out the log probabilities and convert them to probabilities for fresh and rotten.

In [119]:
#vectorizing text into dataframe
cvt      =  CountVectorizer(strip_accents='unicode', binary=True, stop_words='english', ngram_range=(1,1), min_df=2)
X_train2 = pd.DataFrame(cvt.fit_transform(X_train).todense(),
             columns=cvt.get_feature_names())#binary=True if BernoulliNB version of Naive Bayes is used

In [120]:
#model fit
bnb = BernoulliNB()
model = bnb.fit(X_train2, y_train)

In [121]:
#alternative model cross_val direct
cross_val_score(bnb, X_train2, y_train, cv=5)

array([0.7381802 , 0.77430865, 0.7528992 , 0.74564926, 0.75412762])

In [122]:
#vectorize test set, transform directly
X_test2 = pd.DataFrame(cvt.transform(X_test).todense(),
             columns=cvt.get_feature_names())

In [123]:
#score vs baseline 0.61
model.score(X_test2, y_test)

0.7530335474660956

In [1]:
#this is for fresh class, cos didnt encode y
coef_0 = pd.DataFrame(zip(X_train2.columns, model.feature_log_prob_[0]))
coef_0['exp_0'] = np.exp(coef_0[1])*100
coef_0.sort_values('exp_0', ascending=False).head(10)

NameError: name 'pd' is not defined

In [125]:
coef_1 = pd.DataFrame(zip(X_train2.columns, model.feature_log_prob_[1]))
coef_1['exp_1'] = np.exp(coef_1[1])*100
coef_1.sort_values('exp_1', ascending=False).head(10)

,0,1,exp_1
6224,movie,-2.007442,13.433180
3553,film,-2.318846,9.838710
5536,like,-2.709203,6.658986
9111,story,-3.149883,4.285714
5218,just,-3.251666,3.870968
1812,comedy,-3.313035,3.640553
5582,little,-3.405816,3.317972
1532,characters,-3.405816,3.317972
6232,movies,-3.419803,3.271889
4089,good,-3.441156,3.202765


#### 5.B) Make a DataFrame with the probabilities and features.

In [126]:
df_prob = pd.concat([coef_0[[0,'exp_0']], coef_1['exp_1']], axis=1)
df_prob.head(5)

,0,exp_0,exp_1
0,007,0.087311,0.046083
1,10,0.145518,0.391705
2,100,0.101863,0.069124
3,101,0.058207,0.046083
4,10th,0.029104,0.046083


#### 5.C) Create a column that is the difference between the probability of the appearance of fresh and rotten.

In [127]:
#fresh is 0 cos nv encode
df_prob['diff'] = df_prob['exp_0'] - df_prob['exp_1']

#### 5.D) Look at the most likely words for fresh and rotten reviews.

In [128]:
#likely fresh words
df_prob.sort_values('diff', ascending=False).head(10)

,0,exp_0,exp_1,diff
3553,film,14.377183,9.838710,4.538473
930,best,4.132712,1.543779,2.588934
4161,great,2.721187,0.875576,1.845611
6894,performance,2.182771,0.506912,1.675858
3120,entertaining,2.182771,0.622120,1.560651
3560,films,3.827125,2.373272,1.453853
3866,fun,2.473807,1.244240,1.229567
3122,entertainment,1.775320,0.552995,1.222325
418,american,2.022701,0.806452,1.216249
10694,years,1.848079,0.668203,1.179876


In [129]:
#likely rotten words
df_prob.sort_values('diff', ascending=True).head(10)

,0,exp_0,exp_1,diff
5536,like,4.220023,6.658986,-2.438963
767,bad,0.785797,2.557604,-1.771806
5582,little,1.818976,3.317972,-1.498997
2757,doesnt,1.600698,3.064516,-1.463818
6224,movie,12.048894,13.433180,-1.384286
7592,really,0.698487,2.050691,-1.352205
5075,isnt,1.251455,2.511521,-1.260066
7051,plot,1.266007,2.465438,-1.199431
9587,theres,1.775320,2.949309,-1.173989
5218,just,2.706636,3.870968,-1.164332


---

### 6) Examine how your model performs on the testing set.

In [130]:
#score vs baseline 0.61
model.score(X_test2, y_test)

0.7530335474660956

---

### 7) Look at the top 10 movies and reviews likely to be fresh and the top 10 likely to be rotten.

You can fit the model on the full set of data for this.

> **Note:** While it's good at classifying, Naive Bayes is known to be somewhat bad at providing accurate predicted probabilities (beyond getting it on the correct side of 50 percent). It's a good classifier but a bad estimator. 

In [131]:
y_pred = model.predict_proba(X_test2)

In [132]:
y_pred[:,0]

array([0.97892562, 0.89027705, 0.16938948, ..., 0.74412143, 0.03617104,
       0.90330904])

In [133]:
test = pd.DataFrame(X_test).reset_index()
test['fresh_prob'] = y_pred[:,0]

In [134]:
#top 10 likely fresh
test.sort_values('fresh_prob', ascending=False).head(10)['qt'].values

array(['rudy vallee turns in his best performance as a gentle puny millionaire named hackensacker in this brilliant simultaneously tender and scalding 1942 screwball comedy by preston sturges',
       'the most exciting debut in years it is unified by the extraordinary decor  colour supplement chic meets pop art surrealism  which creates a world of totally fantastic reality situated foursquare in contemporary paris',
       'the city of lost children is a stunningly surreal fantasy a fable of longing and danger of heroic deeds and bravery set in a brilliantly realized world of its own it is one of the most audacious original films of the year',
       'where the wild things are is a fiercely innovative film with surprising texture and nuance it captures the joy and exuberance of childhood without shying away from its very real pains and woes',
       'martin scorseses intimate epic about money sex and brute force is a grandly conceived study of what happens to goodfellas from the mean 

In [135]:
#top 10 likely rot
test.sort_values('fresh_prob', ascending=True).head(10)['qt'].values

array(['bland interminable chase scenes take up so much of the story  the hackneyed plot doesnt need much exposition  that the sheer repetitiveness begins to amaze you',
       'as you sit through the interminable twohoursplus that constitute the fifth element  a colossally stupid overbearingly pompous new movie by luc besson  you can expect to become acquainted with boredom on the most elemental level',
       'georgia rule is a bad idea dreadfully executed  on golden pond with fellatio jokes and whimsical incest melodrama and fonda playing her dad who more and more she eerily resembles',
       'with a cigar box of subplots this episodic yarn is more numbing than boring though its increasingly compelling narrative has the illtimed misfortune to collapse completely in its final talky segment',
       'if universal soldier the return isnt the dullest most derivative unimaginative noisy repetitive mindnumbing and generally imbecilic movie of the year its not because director mic rodgers

---

### 8) Find the movies with at least 10 reviews that are most likely to be fresh or rotten.

In [13]:
# A: